In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Cluster1Mondayn.csv to Cluster1Mondayn.csv


In [3]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [4]:
Data = pd.read_csv('Cluster1Mondayn.csv', index_col='Time', parse_dates=True)

In [5]:
data = Data.drop(['dayofyear', 'month', 'dayofweek', 'hourofday', 'minuteofday', 'daytype', 'season'], axis=1)

In [6]:
def split_sequence(sequence, n_steps):
    X, Y = list(), list()
    for i in range(len(sequence)):
        
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x) 
        Y.append(seq_y)
    return np.array(X), np.array(Y)

n_steps = 3
X, Y = split_sequence(data.values, n_steps)

In [7]:
class Energy_Series(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.targets[idx]
        
        return x, y

In [8]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()

        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 64, 1)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(64, 50)

        self.hidden_size = 128
        self.num_layers = 2
        self.lstm = nn.LSTM(1, 128, 2, batch_first=True)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.view(-1, 64)
        x = self.fc1(x)
        x = x.unsqueeze(2)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.fc2(out[:, -1, :])
        return out

In [9]:
model = CNN_LSTM().to(device).float()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [10]:
train = Energy_Series(X.reshape(X.shape[0],X.shape[1],1),Y)
train_loader = torch.utils.data.DataLoader(train,batch_size=2,shuffle=False)

In [11]:
train_losses = []
def Train():
    running_loss = 0
    
    model.train() 
    
    for idx, (inputs,labels) in enumerate(train_loader):
        inputs = inputs.to(device).float()
        labels = labels.to(device).float()
        optimizer.zero_grad()
        preds = model(inputs)
        loss = criterion(preds,labels)
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        
        #del loss, inputs, labels, preds, 
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss)
    
    print(f'train_loss {train_loss}') 

In [12]:
epochs = 500
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()

epochs 1/500
train_loss 0.0175387088794322
epochs 2/500
train_loss 0.012388019205333438
epochs 3/500
train_loss 0.0020222417545081757
epochs 4/500
train_loss 0.0018842501227757447
epochs 5/500
train_loss 0.001873607779970121
epochs 6/500
train_loss 0.001821986314628196
epochs 7/500
train_loss 0.001738999502977261
epochs 8/500
train_loss 0.0016614411978793174
epochs 9/500
train_loss 0.001586282314115019
epochs 10/500
train_loss 0.0015242117595126235
epochs 11/500
train_loss 0.0014652779639169312
epochs 12/500
train_loss 0.0014160470785585214
epochs 13/500
train_loss 0.0013654002385846446
epochs 14/500
train_loss 0.001328104303224626
epochs 15/500
train_loss 0.0012906818899000065
epochs 16/500
train_loss 0.0012588564521380742
epochs 17/500
train_loss 0.0012323465303417148
epochs 18/500
train_loss 0.0012065927797170841
epochs 19/500
train_loss 0.0011849226617126286
epochs 20/500
train_loss 0.0011619028325340521
epochs 21/500
train_loss 0.0011453267276887243
epochs 22/500
train_loss 0.0011

KeyboardInterrupt: ignored

In [27]:
preds = []
batch_size = 1

def Predictions():
    model.eval()
    for i in range(len(X)):
        pred = model(torch.from_numpy(X[i]).unsqueeze(0).to(device).float())
        preds.append(pred.detach().cpu().numpy().item())

In [28]:
Predictions()

In [29]:
predictions = pd.DataFrame(preds, columns=['Energy'])
predictions

,Energy
0,0.233089
1,0.233089
2,0.233089
3,0.233089
4,0.233089
...,...
1816,0.291471
1817,0.290265
1818,0.288146
1819,0.292055


##Metrics


In [30]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [34]:
MSE_CNN_LSTM = MSE(Y, predictions)
RMSE_CNN_LSTM = MSE(Y, predictions)**(0.5)
MAE_CNN_LSTM = MAE(Y, predictions)
MAPE_CNN_LSTM = MAPE(Y, predictions)
R2_CNN_LSTM = r2_score(Y, predictions)

In [35]:
Metric_CNN = {
    'MSE_CNN-LSTM':MSE_CNN_LSTM,
    'RMSE_CNN-LSTM':RMSE_CNN_LSTM,
    'MAE_CNN-LSTM':MAE_CNN_LSTM,
    'MAPE_CNN-LSTM':MAPE_CNN_LSTM,
    'R_Squared_CNN-LSTM':R2_CNN_LSTM,
}

In [36]:
Metric_CNN

{'MAE_CNN-LSTM': 0.013042370644863108,
 'MAPE_CNN-LSTM': 5.974808045040905,
 'MSE_CNN-LSTM': 0.0006731616646988277,
 'RMSE_CNN-LSTM': 0.02594535921313921,
 'R_Squared_CNN-LSTM': 0.9559110699946395}